<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%AA%D8%B9%D8%AF%D9%8A%D9%84%D8%A7%D8%AA_%D8%B9_%D8%A7%D9%84%D8%B1%D8%B3%D9%88%D9%85%D8%A7%D8%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt

def plot_bollinger_bands(data, window=20, num_std_dev=2):
    """رسم بولنجر باندز على مخطط بياني."""
    # حساب المتوسط المتحرك والانحراف المعياري
    data['Middle_Band'] = data['close'].rolling(window=window).mean()
    data['Std_Dev'] = data['close'].rolling(window=window).std()

    # حساب الحد العلوي والحد السفلي
    data['Upper_Band'] = data['Middle_Band'] + (data['Std_Dev'] * num_std_dev)
    data['Lower_Band'] = data['Middle_Band'] - (data['Std_Dev'] * num_std_dev)

    # طباعة القيم
    print("Middle Band:", data['Middle_Band'].iloc[-1])
    print("Upper Band:", data['Upper_Band'].iloc[-1])
    print("Lower Band:", data['Lower_Band'].iloc[-1])

    # رسم المخطط
    plt.figure(figsize=(12, 6))
    plt.plot(data['close'], label='Close Price', color='blue')
    plt.plot(data['Middle_Band'], label='Middle Band', color='orange')
    plt.plot(data['Upper_Band'], label='Upper Band', color='green')
    plt.plot(data['Lower_Band'], label='Lower Band', color='red')
    plt.fill_between(data.index, data['Upper_Band'], data['Lower_Band'], color='lightgray', alpha=0.5)
    plt.title('Bollinger Bands')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

# مثال على كيفية استخدام الدالة
# تأكد من أن لديك بيانات في DataFrame باسم `data` تحتوي على عمود 'close'
# plot_bollinger_bands(data)


In [ ]:
import matplotlib.pyplot as plt

def plot_premium_discount(data):
    """رسم Premium وDiscount على مخطط بياني."""
    # تأكد من أن البيانات تحتوي على الأعمدة المطلوبة
    data['Premium'] = data['close'] - data['low'].rolling(window=20).min()
    data['Discount'] = data['high'].rolling(window=20).max() - data['close']

    # طباعة القيم الأخيرة
    print("Last Premium Value:", data['Premium'].iloc[-1])
    print("Last Discount Value:", data['Discount'].iloc[-1])

    # رسم المخطط
    plt.figure(figsize=(12, 6))
    plt.plot(data.index, data['Premium'], label='Premium', color='blue')
    plt.plot(data.index, data['Discount'], label='Discount', color='red')
    plt.fill_between(data.index, data['Premium'], color='lightblue', alpha=0.5)
    plt.fill_between(data.index, data['Discount'], color='lightcoral', alpha=0.5)
    plt.title('Premium and Discount Bands')
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

# مثال على كيفية استخدام الدالة
# تأكد من أن لديك بيانات في DataFrame باسم `data` تحتوي على الأعمدة 'close', 'low', 'high'
# plot_premium_discount(data)


In [ ]:
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    """حساب مؤشر MACD"""
    data['EMA_short'] = data['close'].ewm(span=short_window, adjust=False).mean()
    data['EMA_long'] = data['close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data

def plot_macd(data):
    """رسم مؤشر MACD."""
    plt.figure(figsize=(12, 6))
    plt.plot(data.index, data['MACD'], label='MACD', color='blue')
    plt.plot(data.index, data['Signal_Line'], label='Signal Line', color='orange')
    plt.axhline(0, color='black', lw=0.5, ls='--')
    plt.title('MACD Indicator')
    plt.xlabel('Date')
    plt.ylabel('MACD Value')
    plt.legend()
    plt.grid()
    plt.show()

# في دالة build_strategy
data = calculate_macd(data)
plot_macd(data)


In [ ]:
import matplotlib.pyplot as plt

def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['close'].iloc[i] < البيانات['close'].iloc[i - 1] and البيانات['close'].iloc[i - 1] > \
                    البيانات['close'].iloc[i - 2]:
                مناطق_طلبات_البيع.append((البيانات.index[i - 1], البيانات['close'].iloc[i - 1]))
            elif البيانات['close'].iloc[i] > البيانات['close'].iloc[i - 1] and البيانات['close'].iloc[i - 1] < \
                    البيانات['close'].iloc[i - 2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i - 1], البيانات['close'].iloc[i - 1]))

        # الحصول على آخر نقطة طلب بيع وشراء
        last_buy_order_block = مناطق_طلبات_الشراء[-1] if مناطق_طلبات_الشراء else None
        last_sell_order_block = مناطق_طلبات_البيع[-1] if مناطق_طلبات_البيع else None

        # طباعة آخر نقاط الطلب
        if last_buy_order_block:
            print(f"آخر نقطة طلب شراء: {last_buy_order_block}")
        else:
            print("لا توجد نقاط طلب شراء.")

        if last_sell_order_block:
            print(f"آخر نقطة طلب بيع: {last_sell_order_block}")
        else:
            print("لا توجد نقاط طلب بيع.")

        # رسم البيانات
        plt.figure(figsize=(12, 6))
        plt.plot(البيانات.index, البيانات['close'], label='سعر الإغلاق', color='blue')

        # رسم مناطق طلب الشراء
        if مناطق_طلبات_الشراء:
            buy_prices = [price for _, price in مناطق_طلبات_الشراء]
            buy_dates = [date for date, _ in مناطق_طلبات_الشراء]
            plt.scatter(buy_dates, buy_prices, color='green', label='مناطق طلب الشراء', marker='^', s=100)

        # رسم مناطق طلب البيع
        if مناطق_طلبات_البيع:
            sell_prices = [price for _, price in مناطق_طلبات_البيع]
            sell_dates = [date for date, _ in مناطق_طلبات_البيع]
            plt.scatter(sell_dates, sell_prices, color='red', label='مناطق طلب البيع', marker='v', s=100)

        plt.title('مناطق طلبات الشراء والبيع')
        plt.xlabel('التاريخ')
        plt.ylabel('السعر')
        plt.legend()
        plt.grid()
        plt.show()

    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")


In [ ]:
import matplotlib.pyplot as plt

def calculate_rsi(data, period=25):
    """حساب RSI ورسمه."""
    try:
        # حساب الفرق بين الأسعار
        delta = data['close'].diff()

        # حساب المكاسب والخسائر
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

        # حساب النسبة RS
        rs = gain / loss

        # حساب RSI
        data['RSI'] = 100 - (100 / (1 + rs))

        # طباعة آخر قيمة من RSI
        print(f"RSI: {data['RSI'].iloc[-1]}")

        # رسم RSI
        plt.figure(figsize=(10, 5))
        plt.plot(data.index, data['RSI'], label='RSI', color='blue')
        plt.axhline(70, linestyle='--', alpha=0.5, color='red')  # مستوى التشبع الشرائي
        plt.axhline(30, linestyle='--', alpha=0.5, color='green')  # مستوى التشبع البيعي
        plt.title('Relative Strength Index (RSI)')
        plt.xlabel('Date')
        plt.ylabel('RSI Value')
        plt.legend()
        plt.grid()
        plt.show()

        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data
